In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, UpSampling2D
from keras.callbacks import ModelCheckpoint

from mne.io import read_raw_brainvision as mne_read
from os.path import join
from os import listdir
import numpy as np

Using TensorFlow backend.


In [3]:
channels_num = 60
window_size = 500
nb_epoch = 50

# Hyper parameters
batch_size = 32
nb_visible = channels_num * window_size
nb_hidden = 1000
nb_emb = 20

corruption_level = 0.001

In [4]:
mne_format_data_dir = join('data', 'resting_state_mne_format')

In [5]:
def create_model():
    inp = Input(shape=(window_size, channels_num, 1))

    encoder = Sequential( [
        Conv2D(128, kernel_size=(5,5), activation='relu', padding='same', input_shape=(window_size,channels_num,1)),
        MaxPooling2D(),
        Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPooling2D(),
        Conv2D(1, kernel_size=(1,1), padding='same'),
    ] )
    print(encoder.summary())
    decoder = Sequential((
        Conv2D(128, kernel_size=(5,5), activation='relu', padding='same', input_shape=(125, 15, 1) ),
        UpSampling2D(),
        Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
        UpSampling2D(),
        Conv2D(1, kernel_size=(1,1), padding='same'),
    ))
    print(decoder.summary())
    autoencoder = Model(inp, decoder(encoder(inp)))
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['mae'])

    print(autoencoder.summary())
    return autoencoder

In [6]:
autoencoder = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 500, 60, 128)      3328      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 250, 30, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 250, 30, 64)       73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 125, 15, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 15, 1)        65        
Total params: 77,185
Trainable params: 77,185
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
#scaler = StandardScaler()
mean, var = None, None

def cut_window(data):
    return data[:, :data.shape[1] - data.shape[1] % window_size]

def read_func(filename, verbose=0):
    d = mne_read(filename, verbose=verbose, preload=True)
    d.drop_channels(['VEOG', 'HEOG', 'STI 014'])
    d = d.get_data()
    return cut_window(np.vstack( [d, np.zeros( (2, d.shape[1]) )]) )

def read_data(data_type, return_batch_np=False):
    global mean, var
    ''' If return_batch_np is False
            return data with shape (None, batch_size, nb_visible)
        else number of batches in data
    '''
    verbose = None if return_batch_np else 0
    
    data_dir = join(mne_format_data_dir, data_type)
    data = np.hstack([read_func( join(data_dir, fn), verbose=verbose) for fn in listdir(data_dir) if fn[-5:] == '.vhdr'])
    
    if mean is None:
        mean, var = data.mean(), data.var()
    #data = scaler.transform(data)
    data = (data - mean) / (var ** (1/2))
    print(data_type, data.mean(), data.var())
    if return_batch_np:
        return data.shape[1] // window_size // batch_size
    return data.T.reshape(-1, window_size, channels_num, 1)

def gen(data_type):
    data = read_data(data_type)
    print(data.shape)
    for ep in range(nb_epoch+1):
        for i in range(data.shape[0] // batch_size):
            x = data[i*batch_size:(i+1)*batch_size, :, :, :]
            yield x, x

In [10]:
train_batches_nb = read_data('train', return_batch_np=True)
val_batches_nb = read_data('validation', return_batch_np=True)

Extracting parameters from data/resting_state_mne_format/train/2505_shirokova_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 606599  =      0.000 ...   606.599 secs...
Extracting parameters from data/resting_state_mne_format/train/miloslavov_22_05_pre_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 603399  =      0.000 ...   603.399 secs...
Extracting parameters from data/resting_state_mne_format/train/gorin_rest_eeg_post_31011200.vhdr...
Setting channel info structure...
Reading 0 ... 603699  =      0.000 ...   603.699 secs...
Extracting parameters from data/resting_state_mne_format/train/2704_zagirova_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 604549  =      0.000 ...   604.549 secs...
Extracting parameters from data/resting_state_mne_format/train/2103_glebko_posteeg.vhdr...
Setting channel info structure...
Reading 0 ... 633899  =      0.000 ...   633.899 secs...
Extracting parameters from data/resting_state_mne_format/train/200317

In [11]:
history = autoencoder.fit_generator(gen('train'), steps_per_epoch=train_batches_nb, epochs=nb_epoch, \
                          validation_data=gen('validation'), validation_steps=val_batches_nb, \
                          #use_multiprocessing=True, \
                          callbacks=[ModelCheckpoint("models/cnn_%s_{epoch:02d}.hdf5"%str(nb_hidden), save_best_only=False)])

Epoch 1/50
train -6.34378883316e-16 1.0
(29879, 500, 60, 1)
932/933 [============================>.] - ETA: 0s - loss: 0.6052 - mean_absolute_error: 0.2604validation -0.164368573828 0.135983946103
(3149, 500, 60, 1)
933/933 [==============================] - 394s - loss: 0.6046 - mean_absolute_error: 0.2603 - val_loss: 0.1057 - val_mean_absolute_error: 0.2577
Epoch 2/50
933/933 [==============================] - 349s - loss: 0.5680 - mean_absolute_error: 0.2470 - val_loss: 0.0953 - val_mean_absolute_error: 0.2452
Epoch 3/50
933/933 [==============================] - 349s - loss: 0.5183 - mean_absolute_error: 0.2336 - val_loss: 0.0901 - val_mean_absolute_error: 0.2411
Epoch 4/50
933/933 [==============================] - 350s - loss: 0.5124 - mean_absolute_error: 0.2287 - val_loss: 0.0903 - val_mean_absolute_error: 0.2402
Epoch 5/50
933/933 [==============================] - 349s - loss: 0.5103 - mean_absolute_error: 0.2263 - val_loss: 0.0916 - val_mean_absolute_error: 0.2418
Epoch 6/50

Exception in thread Thread-51:
Traceback (most recent call last):
  File "/opt/conda/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 560, in data_generator_task
    generator_output = next(self._generator)
StopIteration



StopIteration: 

In [9]:
test_batches_nb = read_data('test', return_batch_np=True)

Extracting parameters from data/resting_state_mne_format/test/zavrin_eyes_closed_eeg_15021500.vhdr...
Setting channel info structure...
Reading 0 ... 302749  =      0.000 ...   302.749 secs...
Extracting parameters from data/resting_state_mne_format/test/zavrin_15021500_eyesclosed_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 319299  =      0.000 ...   319.299 secs...
Extracting parameters from data/resting_state_mne_format/test/300120171600_dagaev_rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 615599  =      0.000 ...   615.599 secs...
Extracting parameters from data/resting_state_mne_format/test/tsoy_pre_eeg_2504.vhdr...
Setting channel info structure...
Reading 0 ... 652699  =      0.000 ...   652.699 secs...
test 5.32180965766e-17 1.0


In [10]:
from keras.models import load_model
autoencoder = load_model('models/cnn_1000_06.hdf5')

In [11]:
autoencoder.evaluate_generator(gen('test'), steps=test_batches_nb)

test 5.32180965766e-17 1.0
(3779, 500, 60, 1)


[0.60504258853399151, 0.58064251835063352]

In [12]:
autoencoder.metrics_names

['loss', 'mean_absolute_error']